# PHASE 2: PROFIT

In [1]:
import utils.dir_utils as dir_utils
import utils.dict_utils as dict_utils
import utils.ptr_utils as ptr_utils
import utils.constants as constants 
import utils.profit as profit 
import helpers.official as official
import utils.csv_utils as csv_utils
import copy

_, input_df = dir_utils.get_data(combined=True)
_, house_input_df = dir_utils.get_data(house=True)
_, senate_input_df = dir_utils.get_data(senate=True)

### Possible profit?

In [2]:
def setup():
    purchases = set()
    sales = set()

    for _,t in input_df.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            core =  (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
            if t[constants.TYPE] == 'Purchase': 
                purchases.add(core)
            elif t[constants.TYPE] == 'Sale (Full)' or t[constants.TYPE] == 'Sale (Partial)':
                sales.add(core)

    return purchases, sales 

def match(purchases, sales):
    sales_copy = copy.deepcopy(sales)

    # { (name, ticker, tdate_sale) : [(share_diff, tdate_purch), ... , ... ]  }
    possible_matches = {}
    i = 0 
    for (tdate_sale, name_sale, ticker_sale) in sales_copy: 
        i += 1 
        print(i)   
        for (tdate_purch, name_purch, ticker_purch)  in purchases:
            if name_purch == name_sale and ticker_purch == ticker_sale and ptr_utils.is_this_date_after(tdate_sale, tdate_purch):
                diff = profit.share_diff(ticker_purch, tdate_sale, tdate_purch)
                if diff: 
                    possible_matches = dict_utils.increment_list_in_dictionary(possible_matches, (name_sale, ticker_sale, tdate_sale), (diff, tdate_purch))
            
    return possible_matches, purchases, sales 

def stablize(p):
    # { (name, ticker) : {tdate_sale: (143.2, '2019/11/05'), '2015/07/09': (28.95, '2020/03/26'), '2017/12/07': -1, '2021/01/11': (-2.02, '2021/01/15')} }
    res = {}
    for i in p:
        m = profit.stableMarriage(p[i])
        res = dict_utils.increment_dictionary(res, i, m, not_math=True)    
    return res 

def output(res):
    csv_output = []
    for ((name, ticker), dict) in res.items():
        for buy_date, possible_match in dict.items():
            if possible_match != -1:
                (diff, sell_date) = possible_match
                csv_output.append([name, ticker, buy_date, diff, sell_date])

    dir = dir_utils.makesubdir(constants.path_csv, constants.PROFIT)
    csv_utils.make_csv_base(dir, constants.PROFIT, [constants.OFFICIAL, constants.TICKER, constants.BUYDATE, constants.PROFIT, constants.SELLDATE], csv_output)
    return res 

p, s = setup()
print('len(sales): {}'.format(len(s)))
print('len(purchases): {}'.format(len(p)))
pm, p, s = match(p, s)
p = profit.get_instances(pm)
res = stablize(p)
output(res)

# 14 hours to run. 

len(sales): 8171
len(purchases): 8564
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
